In [3]:
import pandas as pd
import numpy as np
import requests
import ast 
import datetime

En este ejercicio, tendréis que crear una clase con el código que usamos en los ejercicios de pair programming de ETL Transformación I y II.

In [ ]:
paises = {"USA" : [39.7837304,-100.445882], "AUSTRALIA" : [-24.7761086, 134.755], "SOUTH AFRICA" : 
    [-28.8166236, 24.991639], "NEW ZEALAND" : [-41.5000831, 172.8344077], "PAPUA NEW GUINEA" : [-5.6816069, 144.2489081]}
producto = "civil"
producto2 = "astro"


In [ ]:
class Extraccion: 
    # primero tenemos que crear el constructor con las variables globales que usaremos. Están son variables que podrán cambiar cada vez que llamemos a la clase. 
    def __init__(self, paises):

        # definimos cada una de las variables. Recordamos que tenemos que usar el método .self para definirlas y que la clase entienda que las podremos usar en otros métodos de la clase. 

        self.paises = paises

    # definimos nuestro primer método. El que nos hará la llamada a la API
    # Si comparamos, veremos que la función es exactamente igual a la que hicimos en la clase anterior, solo que con unos pequeños cambios. En este caso incluimos el parámetro self
    def llamada_API(self, producto):

        self.producto = producto

        df_final = pd.DataFrame()

        for key, value in paises.items():
        
            url = f'http://www.7timer.info/bin/api.pl?lon=-{value[1]}&lat={value[0]}&product={producto}&output=json'

            response = requests.get(url=url)
            codigo_estado = response.status_code
            razon_estado = response.reason
            if codigo_estado == 200:
                print('La peticion se ha realizado correctamente, se ha devuelto el código de estado:',codigo_estado,' y como razón del código de estado: ',razon_estado)
            elif codigo_estado == 402:
                print('No se ha podido autorizar usario, se ha devuelto el código de estado:', codigo_estado,' y como razón del código de estado: ',razon_estado)
            elif codigo_estado == 404:
                print('Algo ha salido mal, el recurso no se ha encontrado,se ha devuelto el código de estado:', codigo_estado,' y como razón del código de estado: ',razon_estado)
            else:
                print('Algo inesperado ha ocurrido, se ha devuelto el código de estado:', codigo_estado,' y como razón del código de estado: ',razon_estado)
            
            df = pd.json_normalize(response.json()["dataseries"])
            df['latitud'] = value[0]
            df['longitud'] = value[1]
            df['pais'] = key
            df_final = pd.concat([df_final,df], axis=0, ignore_index=True) 
         
        
    
    # definimos un nuevo método, el de limpiar los datos obtenidos de la llamada a la API para el producto civil. 
    def limpiar_civil(self, df): 

        self.df = df

        # lo primero que tenemos que hacer es crear la columna de fecha en el dataframe nuevo 
        # En este caso será la fecha del día de hoy. 
        hoy = datetime.now()
        hoy = datetime.strftime(hoy, '%Y-%m-%d')

        # creamos la nueva columna
        df["fecha"] = hoy

        return

        # de nuevo usamos el self para crear las nuevas columnas en función de los parámetros pasados al definir la clase. Recordamos, el caminito de baldosas amarillas.

    
    # creamos una clase para limpiar el resultado de la llamada a la API para el producto astro
    def limpiar_astro(self, df):

        #seleccionamos solo las columnas que nos interesan
        df = df[["seeing", "transparency", "timepoint"]]

        # creamos la columna de fecha: 
        hoy = datetime.now()
        hoy = datetime.strftime(hoy, '%Y-%m-%d')
        df["fecha"] = hoy


        # insertamos las columnas de la localidad
    
        df["ciudad"] = self.ciudad

        return df
    
    # de nuevo, hacemos lo mismo que en la lección anterior. 
    def juntar_dfs(self, df_completo, df_civil, df_visibilidad): 

        df_hoy = pd.merge(df_civil , df_visibilidad , on=['fecha', "timepoint"], how = "inner")
        print("El df de hoy es ", df_hoy.columns)
        print("-----------------------------------------")
        print("El df de completo es ", df_completo.columns)
        # lo primero que hacemos es concatenar los dataframes con la información general. df_completo y df_civil
        df_completo = pd.concat([df_completo, df_hoy], axis = 0)

        # ahora es el turno de unir el dataframe con la información de la visibilidad con el completo
        # en este caso el how lo ponemos como left ya que queremos que se quede con toda la info que en la primera tabla que le pasamos que es la que tiene toda la información
        #df_completo = pd.merge(df_completo , df_visibilidad , on=['fecha', "timepoint"], how = "inner")

        # guardamos los datos
        df_completo.to_pickle('datos/datos_actualizados.pkl')
        df_completo.to_csv('datos/datos_actualizados.csv')

        return df_completo
    
    

    def chequear_datos(self, df): 
    
        print("Las columnas son:", "\n")
        print(list(df.columns))
        print("-----------------------------------------")

        print("Los tipos de datos que tenemos son:", "\n")
        print(df.dtypes)
        print("-----------------------------------------")

        print("El porcentaje de nulos:", "\n")
        print((df.isnull().sum() / df.shape[0]) *  100)

    def limpiar_dataframe(self, df, lista_columnas): 

        #convertimos la fecha a datetime
        df["fecha"] = pd.to_datetime(df["fecha"])

        # reemplazamos los nulos de las columnas por la media
         # lista de columnas en las que queremos reemplazar los nulos
        df[lista_columnas]=df[lista_columnas].fillna(df.mean())

        # quitar columnas repetidas:
        
        df.drop(["ciudad_y"], axis = 1, inplace = True)
        
        # renombrar columnas
        
        df.rename(columns = {"ciudad_x": "ciudad"}, inplace = True)

        # quitar % 
        df["rh2m"] = df["rh2m"].replace(r"%", "", regex = True)

        # guardamos los datos una vez limpios
        df.to_pickle('datos/datos_Madrid.pkl')
        df.to_csv('datos/datos_Madrid.csv')

        return df
